In [50]:
import pandas as pd # manejo de datos
import numpy as np # manejo de arreglos tablas y matrices
import matplotlib.pyplot as plt # libreria para graficar
import seaborn as sns # Graficar series de tiempo 
import folium
import geopandas as gpd
import folium 
from folium.plugins import HeatMap

In [51]:
# Guardar y leer el archivo de GeoJson
worldgeo = gpd.read_file('world_countries.json')

# Filtrar el GeoDataFrame para incluir solo los países de interés
countries_of_interest = {'United Kingdom', 'France', 'Italy', 'Germany'}
worldgeo = worldgeo[worldgeo['name'].isin(countries_of_interest)]

# Renombrar la columna "name" a "Country"
worldgeo = worldgeo.rename(columns={'name': 'Country'})

worldgeo = worldgeo.set_index("Country")
worldgeo = worldgeo.drop(columns=["id"])

# Mostrar los primeros registros (¡sin eliminar geometry!)
worldgeo.head()

,geometry
Country,
Germany,"POLYGON ((9.92191 54.9831, 9.93958 54.59664, 1..."
France,"MULTIPOLYGON (((-52.55642 2.5047, -52.93966 2...."
United Kingdom,"MULTIPOLYGON (((-5.66195 54.5546, -6.19788 53...."
Italy,"MULTIPOLYGON (((15.52038 38.23116, 15.16024 37..."


In [52]:
# Guarda lung_cancer_prediction_dataset.csv
lung_cancer = pd.read_csv('lung_cancer_prediction_dataset.csv')
lung_cancer.head()

# Estandarizar los nombres de los países en lung_cancer para que coincidan con worldgeo
lung_cancer['Country'] = lung_cancer['Country'].replace({'UK': 'United Kingdom'})

# Filtrar el DataFrame para incluir solo los países de interés Unitwed Kingdom, France, Italy, Germany
lung_cancer = lung_cancer[lung_cancer['Country'].isin(countries_of_interest)]

#Eliminamos las columnas que no sean Annual_Lung_Cancer_Deaths y Country
lung_cancer = lung_cancer[['Country','Annual_Lung_Cancer_Deaths']]

# Agrupar por país y sumar las muertes anuales por cáncer de pulmón
lung_cancer_grouped = lung_cancer.groupby('Country', as_index=False)['Annual_Lung_Cancer_Deaths'].sum()

# Unir los datos
merged_data = worldgeo.merge(lung_cancer_grouped, on='Country')

# Mostrar el resultado
merged_data.head()


,Country,geometry,Annual_Lung_Cancer_Deaths
0,Germany,"POLYGON ((9.92191 54.9831, 9.93958 54.59664, 1...",392715000
1,France,"MULTIPOLYGON (((-52.55642 2.5047, -52.93966 2....",294360000
2,United Kingdom,"MULTIPOLYGON (((-5.66195 54.5546, -6.19788 53....",303940000
3,Italy,"MULTIPOLYGON (((15.52038 38.23116, 15.16024 37...",272862000


In [54]:
import folium

# Crear el mapa centrado en Europa
m = folium.Map(location=[50, 10], zoom_start=4)

# Crear el mapa coroplético (coloreado por país)
folium.Choropleth(
    geo_data=merged_data,  # Datos geoespaciales con geometría y valores
    name="Choropleth",
    data=merged_data,  # Datos numéricos
    columns=['Country', 'Annual_Lung_Cancer_Deaths'],  # Columnas de interés
    key_on="feature.properties.Country",  # Clave para unir con GeoJSON
    fill_color="YlOrRd",  # Esquema de color
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Muertes Anuales por Cáncer de Pulmón"
).add_to(m)

# Agregar control de capas
folium.LayerControl().add_to(m)

# Guardar el mapa en un archivo HTML
m.save("mapa_cloropleth.html")

# Mostrar el mapa
m
